In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **I. TIỀN XỬ LÝ DỮ LIỆU**

In [ ]:
# Đọc dữ liệu 4 bảng user, context, mobile_plan_user, mobile_plan_attr của tập train
user = pd.read_csv('user.csv')
context = pd.read_csv('context.csv')
mobile_plan_user = pd.read_csv('mobile_plan_user.csv')
mobile_plan_attr = pd.read_csv('mobile_plan_attr.csv')

## **1. Bảng user**

In [ ]:
user.head()

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,tik_freq,use_less_than_2GB,use_2GB_to_4GB
0,11156,Rachel Gibbs,Female,21,NaN,Unemployed,39100$,Unmarrie d_2,AUSTRALIA,(08)-8012-7556,Astronomer,0,0,9,8,3
1,4297,Karen Anderson,Other,22,NaN,Unemployed,41000$,U nmarr ied_2,ENGLAND,943-646-5203,Air cabin crew,0,0,23,7,2
2,13301,김지원,Female,24,NaN,Unemployed,44300$,Unma rried_1,KOREA,010-4500-9888,기계공학 기술자 및 연구원,0,0,23,5,2
3,9920,Elisabeth Wähner,Female,24,NaN,Unemployed,44400$,Un ma rried_1,DENMARK,+49(0) 587406963,Medizininformatiker,0,0,21,7,2
4,8424,Sra. Maria Luiza Nogueira,Female,25,NaN,Unemployed,1100000000 VND,U nmarr ied_1,BRAZIL,(084) 0568 1445,Ufólogo,0,0,22,5,3


In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11572 entries, 0 to 11571
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 11572 non-null  int64 
 1   name               11572 non-null  object
 2   gender             11572 non-null  object
 3   age                11572 non-null  int64 
 4   education          7569 non-null   object
 5   profession         11572 non-null  object
 6   income             11572 non-null  object
 7   living_with        11572 non-null  object
 8   nation             11572 non-null  object
 9   phone              11572 non-null  object
 10  job                11572 non-null  object
 11  fb_freq            11572 non-null  int64 
 12  yt_freq            11572 non-null  int64 
 13  tik_freq           11572 non-null  int64 
 14  use_less_than_2GB  11572 non-null  int64 
 15  use_2GB_to_4GB     11572 non-null  int64 
dtypes: int64(7), object(9)
memory usage: 1.4

### **1.1. Xử lý NULL & DUPLICATE**

In [ ]:
# Đếm số lượng null của mỗi cột
user.isnull().sum()

,0
id,0
name,0
gender,0
age,0
education,4003
profession,0
income,0
living_with,0
nation,0
phone,0


**Remark:** Chỉ có cột **education** có **4003** giá trị NULL

-> Thay thế các giá trị NULL bằng **"Unknown"**

In [ ]:
# Thay thế giá trị NULL trong cột "education" bằng "Unknown"
user['education'] = user['education'].fillna('Unknown')

In [ ]:
# Check lại
user.isnull().sum()

,0
id,0
name,0
gender,0
age,0
education,0
profession,0
income,0
living_with,0
nation,0
phone,0


In [ ]:
# Đếm số lượng duplicate :
user.duplicated().sum()

np.int64(0)

### **1.2. Tối ưu và chuẩn hóa**

**Income:** gồm hai đơn vị tiền là $ và VND, đưa về cùng
một đơn vị tiền là **VND**, giả định tỷ giá là **24000**.

In [ ]:
# Unique value của Income
user['income'].value_counts()

,count
income,
852500000 VND,17
46200$,16
25900$,16
1312500000 VND,16
32300$,15
...,...
131600$,1
191900$,1
3410000000 VND,1


In [ ]:
# Loại bỏ khoảng trắng thừa và dấu phẩy
user['income'] = user['income'].str.replace(',', '', regex=False).str.strip()

# Chuyển từng giá trị thành số VND
income_list = []
for val in user['income']:
    val = val.strip()  # bỏ khoảng trắng giữa giá trị số và VND/USD
    if '$' in val:     # nếu là USD
        number = float(val.replace('$','').strip())
        number = number * 24000
        income_list.append(number)
    elif 'VND' in val: # nếu là VND
        number = float(val.replace('VND','').strip())
        income_list.append(number)
    else:              # nếu không có đơn vị
        income_list.append(float(val))

# Gán lại giá trị cho cột Income
user['income'] = income_list

# Kiểm tra kết quả
user

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,tik_freq,use_less_than_2GB,use_2GB_to_4GB
0,11156,Rachel Gibbs,Female,21,Unknown,Unemployed,9.384000e+08,Unmarrie d_2,AUSTRALIA,(08)-8012-7556,Astronomer,0,0,9,8,3
1,4297,Karen Anderson,Other,22,Unknown,Unemployed,9.840000e+08,U nmarr ied_2,ENGLAND,943-646-5203,Air cabin crew,0,0,23,7,2
2,13301,김지원,Female,24,Unknown,Unemployed,1.063200e+09,Unma rried_1,KOREA,010-4500-9888,기계공학 기술자 및 연구원,0,0,23,5,2
3,9920,Elisabeth Wähner,Female,24,Unknown,Unemployed,1.065600e+09,Un ma rried_1,DENMARK,+49(0) 587406963,Medizininformatiker,0,0,21,7,2
4,8424,Sra. Maria Luiza Nogueira,Female,25,Unknown,Unemployed,1.100000e+09,U nmarr ied_1,BRAZIL,(084) 0568 1445,Ufólogo,0,0,22,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11567,11602,Austin Barber,Male,26,Bachelor,Sales & Related,1.968000e+09,Single_0,ENGLAND,001-798-297-3345x825,Publishing rights manager,0,0,3,5,3
11568,5645,남상철,Male,26,Bachelor,Sales & Related,2.013600e+09,Singl e_0,KOREA,043-900-0635,캐드원,0,0,2,4,1
11569,15477,John Blanchard,Male,30,Bachelor,Sales & Related,1.838400e+09,S ing le_0,US,230-283-6789x89147,"Designer, industrial/product",0,0,3,7,3
11570,8885,Troy Green,Male,30,Bachelor,Sales & Related,2.087500e+09,Si n gle_0,AUSTRALIA,+61.472.119.694,Patent examiner,0,0,2,7,3


**living_with**

In [ ]:
# Unique value của cột living_with
user['living_with'].value_counts()

,count
living_with,
Single_0,756
Unmarried_0,324
Married_1,302
Married_0,251
Sing le_0,170
...,...
S ing le_3,1
U nma rried_4,1
Wi d o wed_4,1


**Remark**:
- Một số dòng có khoảng trắng thừa giữa chữ.


In [ ]:
# Thay thế khoảng trắng trong chuỗi bằng rỗng:
user['living_with'] = user['living_with'].str.replace(' ', '')

# Kiểm tra lại unique values của cột living_with sau khi strip
user['living_with'].value_counts()

,count
living_with,
Single_0,3964
Unmarried_0,1591
Married_1,1526
Married_0,1408
Married_2,793
Married_4,420
Married_3,377
Unmarried_1,261
Single_1,239


**Remark:**
- Số thứ 2 sau _ là số con (Number_child), số thứ nhất là trạng thái hôn nhân (Marriage_status).

=> Tách cột Living_with thành 2 cột Marriage_status và Number_child để thuận tiện cho việc phân tích.

In [ ]:
user[['marriage_status','number_child']] = user['living_with'].str.split('_', expand=True)

user['number_child'] = user['number_child'].astype(int)

In [ ]:
user

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,tik_freq,use_less_than_2GB,use_2GB_to_4GB,marriage_status,number_child
0,11156,Rachel Gibbs,Female,21,Unknown,Unemployed,9.384000e+08,Unmarried_2,AUSTRALIA,(08)-8012-7556,Astronomer,0,0,9,8,3,Unmarried,2
1,4297,Karen Anderson,Other,22,Unknown,Unemployed,9.840000e+08,Unmarried_2,ENGLAND,943-646-5203,Air cabin crew,0,0,23,7,2,Unmarried,2
2,13301,김지원,Female,24,Unknown,Unemployed,1.063200e+09,Unmarried_1,KOREA,010-4500-9888,기계공학 기술자 및 연구원,0,0,23,5,2,Unmarried,1
3,9920,Elisabeth Wähner,Female,24,Unknown,Unemployed,1.065600e+09,Unmarried_1,DENMARK,+49(0) 587406963,Medizininformatiker,0,0,21,7,2,Unmarried,1
4,8424,Sra. Maria Luiza Nogueira,Female,25,Unknown,Unemployed,1.100000e+09,Unmarried_1,BRAZIL,(084) 0568 1445,Ufólogo,0,0,22,5,3,Unmarried,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11567,11602,Austin Barber,Male,26,Bachelor,Sales & Related,1.968000e+09,Single_0,ENGLAND,001-798-297-3345x825,Publishing rights manager,0,0,3,5,3,Single,0
11568,5645,남상철,Male,26,Bachelor,Sales & Related,2.013600e+09,Single_0,KOREA,043-900-0635,캐드원,0,0,2,4,1,Single,0
11569,15477,John Blanchard,Male,30,Bachelor,Sales & Related,1.838400e+09,Single_0,US,230-283-6789x89147,"Designer, industrial/product",0,0,3,7,3,Single,0
11570,8885,Troy Green,Male,30,Bachelor,Sales & Related,2.087500e+09,Single_0,AUSTRALIA,+61.472.119.694,Patent examiner,0,0,2,7,3,Single,0


**nation**

In [ ]:
# Unique value của nation
user['nation'].value_counts()

,count
nation,
US,2980
ENGLAND,2934
CHINA,955
JAPAN,953
RUSSIA,943
BRAZIL,729
AUSTRALIA,718
KOREA,707
DENMARK,653


**Remark:**
- Đặc trưng Nation được đổi thành dạng chữ title case để đồng nhất
với các đặc trưng còn lại.

In [ ]:
# Chuyển cột 'nation' thành Title Case
user['nation'] = user['nation'].str.title()

# Kiểm tra số lượng mỗi giá trị sau khi chuyển
nation_counts = user['nation'].value_counts()
print(nation_counts)

nation
Us           2980
England      2934
China         955
Japan         953
Russia        943
Brazil        729
Australia     718
Korea         707
Denmark       653
Name: count, dtype: int64


**job**

In [ ]:
user['job'].value_counts()

,count
job,
其他,56
Artist,38
検察官,29
Krankenschwester,29
高等学校教員,28
...,...
法务主管/专员,1
产品总监,1
Modelista,1


In [ ]:
# Cài đặt thư viện googletrans
!pip install googletrans==4.0.0-rc1

In [ ]:
from googletrans import Translator

In [ ]:
translator = Translator()

# Lấy danh sách job duy nhất
unique_jobs = user['job'].unique()

# Dịch từng job duy nhất
translated_dict = {}
for job in unique_jobs:
    try:
        translated_text = translator.translate(job, src='auto', dest='en').text
        translated_dict[job] = translated_text
    except:
        translated_dict[job] = job  # giữ nguyên nếu lỗi

# Map kết quả dịch về cột gốc
user['job'] = user['job'].map(translated_dict)

print(user)

          id                       name  gender  age education  \
0      11156               Rachel Gibbs  Female   21   Unknown   
1       4297             Karen Anderson   Other   22   Unknown   
2      13301                        김지원  Female   24   Unknown   
3       9920           Elisabeth Wähner  Female   24   Unknown   
4       8424  Sra. Maria Luiza Nogueira  Female   25   Unknown   
...      ...                        ...     ...  ...       ...   
11567  11602              Austin Barber    Male   26  Bachelor   
11568   5645                        남상철    Male   26  Bachelor   
11569  15477             John Blanchard    Male   30  Bachelor   
11570   8885                 Troy Green    Male   30  Bachelor   
11571  12092              Joshua Nelson    Male   27  Bachelor   

            profession        income  living_with     nation  \
0           Unemployed  9.384000e+08  Unmarried_2  Australia   
1           Unemployed  9.840000e+08  Unmarried_2    England   
2           Une

In [ ]:
user['job'].value_counts()

,count
job,
other,56
Nurse,55
Chemist,50
Artist,39
prosecutor,32
...,...
Boilermaker,1
Escort,1
physical and occupational therapist,1


In [ ]:
unique_set = set(user['job'])
print(unique_set)

{'Veterinarian', 'laboratory technician', 'Side gunner', 'New energy electronic control engineer', 'assembler', 'Financial planner', 'Marine scientist', '广告创意/设计经理', 'Public librarian', 'Advice worker', 'Exhaust gas treatment engineer', 'Историк', '钢筋工', 'Investment banker, operational', 'painter and sculptor', 'Coppersmith', '일반기계 조립원', 'Flight radio operator', '硫化工', 'Train/subway driver', 'Physical education professional', 'Oftalmologista', 'Landscaper', 'Confeiteiro', 'Broker', 'Forensic scientist', '장기 부사관 및 준위', 'child care assistant', 'YouTuber', '가구 제조 및 수리원', 'medical clerk', 'Industrial maintenance professional', 'Табаковод', 'Winch driver', '机电工程师', 'spanish translation', 'Image maker', 'warehouse manager', '하역 및 적재 단순 종사원', '客服总监', '기타 사육관련 종사원', '高等学校教員', 'Data Communications Engineer', '网络信息安全工程师', 'Real estate project bidding', 'Authors and related experts', '专科医生', 'Cabin crew', 'Research scientist (medical)', 'Collector', '财务总监', 'Educator integrated into public health

## **2. Bảng context**

### **2.1. Xử lý NULL & DUPLICATE**

In [ ]:
# Hiện info của bảng
context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11572 entries, 0 to 11571
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            11572 non-null  int64 
 1   purpose       11572 non-null  object
 2   go_with       11572 non-null  object
 3   weather       11572 non-null  object
 4   time          11572 non-null  object
 5   viettel_no_0  11572 non-null  int64 
 6   viettel_no_1  11572 non-null  int64 
 7   viettel_no_2  11572 non-null  int64 
 8   to_hanoi      11572 non-null  int64 
 9   to_other      11572 non-null  int64 
 10  score         11572 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 994.6+ KB


In [ ]:
# Null của bảng:
context.isnull().sum()

,0
id,0
purpose,0
go_with,0
weather,0
time,0
viettel_no_0,0
viettel_no_1,0
viettel_no_2,0
to_hanoi,0
to_other,0


In [ ]:
# Kiểm tra duplicate:
context.duplicated().sum()

np.int64(0)

**Remark**
- Bảng này không có null, duplicate

### **2.2. Tối ưu và chuẩn hóa**

**go_with**

In [ ]:
context['go_with'].value_counts()

,count
go_with,
Alone,1333
Friend(s),616
Family,384
A~lone,65
Alo ne,57
...,...
?Fri&end?(s),1
F~ri%end(s),1
A%lone,1


In [ ]:
import re

# Hàm để làm sạch dữ liệu
def normalize_go_with(text):
    if pd.isna(text):
        return text

    # Bước 1: Dùng Regex thay thế tất cả ký tự KHÔNG phải chữ cái (a-z, A-Z) thành rỗng
    # Điều này sẽ xóa cả dấu câu, ký tự đặc biệt và khoảng trắng thừa
    clean_text = re.sub(r'[^a-zA-Z]', '', str(text))

    # Bước 2: Chuẩn hóa viết hoa chữ đầu (Title Case)
    # Kết quả lúc này sẽ là: Alone, Family, Friends
    clean_text = clean_text.capitalize()

    return clean_text

# 3. Áp dụng hàm vào cột
context['go_with'] = context['go_with'].apply(normalize_go_with)

In [ ]:
context['go_with'].value_counts()

,count
go_with,
Alone,6679
Friends,3012
Family,1881


**weather**

In [ ]:
context['weather'].value_counts()

,count
weather,
Sunny,1873
Snowy,257
Rainy,237
Sun!ny,100
Sun?ny,89
...,...
S?~&&nowy,1
S!!n%owy,1
%&?R?ainy,1


In [ ]:
import re

# Hàm để làm sạch dữ liệu
def normalize_weather(text):
    if pd.isna(text):
        return text

    # Bước 1: Dùng Regex thay thế tất cả ký tự KHÔNG phải chữ cái (a-z, A-Z) thành rỗng
    # Điều này sẽ xóa cả dấu câu, ký tự đặc biệt và khoảng trắng thừa
    clean_text = re.sub(r'[^a-zA-Z]', '', str(text))

    # Bước 2: Chuẩn hóa viết hoa chữ đầu (Title Case)
    # Kết quả lúc này sẽ là: Alone, Family, Friends
    clean_text = clean_text.capitalize()

    return clean_text

# 3. Áp dụng hàm vào cột
context['weather'] = context['weather'].apply(normalize_weather)

In [ ]:
context['weather'].value_counts()

,count
weather,
Sunny,9189
Snowy,1282
Rainy,1101


**time**

In [ ]:
time_counts = context['time'].value_counts()
time_counts

,count
time,
7:00,751
6:00,735
6:00 AM,483
7:00 AM,481
17:00,383
...,...
7PM,19
1PM,18
9PM,18


In [ ]:
import re
import pandas as pd

def convert_time(t):
    if pd.isna(t):
        return t
    t = str(t).strip()

    # Thêm khoảng trắng nếu dạng '7PM' → '7 PM'
    t = re.sub(r'(\d)([APap][Mm])$', r'\1 \2', t)

    # Thử parse nhiều định dạng phổ biến
    for fmt in ['%I %p', '%I:%M %p', '%H:%M', '%H']:
        try:
            return pd.to_datetime(t, format=fmt).strftime('%H:00')
        except:
            continue

    # Nếu không parse được → giữ nguyên
    return t

# Áp dụng cho cột time
context['time'] = context['time'].apply(convert_time)


In [ ]:
context['time'].value_counts()

,count
time,
07:00,1447
06:00,1442
17:00,766
15:00,749
16:00,715
08:00,711
18:00,709
10:00,700
09:00,684


**viettel_no_0 & viettel_no_1 & viettel_no_2**

In [ ]:
# Lấy dữ liệu 3 cột viettel_no_0 viettel_no_1 viettel_no_2
viettel_no = context[['viettel_no_0', 'viettel_no_1', 'viettel_no_2']]

In [ ]:
viettel_no

,viettel_no_0,viettel_no_1,viettel_no_2
0,1,0,0
1,1,1,0
2,1,1,0
3,1,1,0
4,1,1,0
...,...,...,...
11567,1,1,0
11568,1,1,0
11569,1,0,0
11570,1,1,1


**Remarks:**

- Ba biến **viettel_no_0**, **viettel_no_1** và **viettel_no_2** được dùng để mô tả thời điểm khách được giới thiệu SIM Viettel **(15’, 30’, 45’)**. Tuy nhiên, dữ liệu gốc được ghi theo dạng **tích lũy** (ví dụ: nếu được giới thiệu trong 45 phút đầu thì cả ba cột đều bằng 1).
- Để **phản ánh đúng ý nghĩa và tránh chồng lấn giữa các mức thời gian**, ta chuẩn hóa lại ba cột này sang dạng **one-hot encoding**: mỗi khách chỉ thuộc đúng một mức thời gian duy nhất.

In [ ]:
def fix_viettel(row):
    # Nếu 45 phút (viettel_no_2 = 1) → hai cột còn lại = 0
    if row['viettel_no_2'] == 1:
        return 0, 0, 1

    # Nếu 30 phút (viettel_no_1 = 1) → hai cột còn lại = 0
    elif row['viettel_no_1'] == 1:
        return 0, 1, 0

    # Nếu 15 phút (viettel_no_0 = 1) → hai cột còn lại = 0
    elif row['viettel_no_0'] == 1:
        return 1, 0, 0

    # Nếu cả 3 đều 0 → giữ nguyên 0-0-0
    else:
        return 0, 0, 0

# Áp dụng cho toàn bộ dữ liệu
context[['viettel_no_0','viettel_no_1','viettel_no_2']] = context.apply(fix_viettel, axis=1, result_type='expand')

**to_hanoi & to_other**

In [ ]:
# Check có dòng nào cột to_hanoi và to_other bằng 1 không?
context[(context['to_hanoi'] == 1) & (context['to_other'] == 1)]

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score


**Remarks:** Không có sự mâu thuẫn giữa việc đi **Hà Nội** và đi **nơi khác**. Một người không thể đi cùng lúc 2 nơi được.

## 3.Bảng mobile_plan_user

### 3.1. Xử lý NULL & DUPLICATE

In [ ]:
mobile_plan_user.head(5)

,id,mobile_plan,accept
0,11156.0,DATASILVER,1.0
1,4297.0,SOCIALMEDIAGOLD,1.0
2,13301.0,DATASILVER,1.0
3,9920.0,SOCIALMEDIAGOLD,1.0
4,8424.0,DATASILVER,1.0


In [ ]:
# Duplicate
mobile_plan_user.duplicated().sum()

np.int64(7385)

In [ ]:
# Xóa các dòng bị duplicate
mobile_plan_user.drop_duplicates(inplace=True)
# Check lại
mobile_plan_user.duplicated().sum()

np.int64(0)

In [ ]:
mobile_plan_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37936 entries, 0 to 45320
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           37936 non-null  float64
 1   mobile_plan  35481 non-null  object 
 2   accept       35400 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.2+ MB


In [ ]:
# Đếm số lượng NULL
mobile_plan_user.isnull().sum()

,0
id,0
mobile_plan,2455
accept,2536


In [ ]:
# Tính tỷ lệ null mỗi cột
mobile_plan_user.isnull().mean()

,0
id,0.000000
mobile_plan,0.064714
accept,0.066849


**Remark:**

- Đây là dữ liệu hành vi khách hàng, không thể suy đoán accept=1/0.

- Thay bằng “Unknown” không hợp lý vì sẽ tạo sai lệch khi thống kê.

- Tỷ lệ null ~6% là thấp → bỏ đi không ảnh hưởng chất lượng phân tích.

->  Việc này giúp đảm bảo tính chính xác của các biến tổng hợp và giữ dữ liệu ở trạng thái hoàn chỉnh trước khi phân tích.


In [ ]:
# Drop NULL
mobile_plan_user.dropna(inplace=True)
# Check lại
mobile_plan_user.isnull().sum()

,0
id,0
mobile_plan,0
accept,0


### 3.2. Tối ưu và chuẩn hóa

In [ ]:
# In ra các dòng có ID = 1001
mobile_plan_user[mobile_plan_user['id'] == 1001]

,id,mobile_plan,accept
3087,1001.0,SOCIALMEDIA,1.0
18162,1001.0,SOCIALMEDIA,0.0
18163,1001.0,DATACALL,0.0


**Remarks:**
- Do bảng **mobile_plan_user** có nhiều dòng cho mỗi khách hàng (mỗi dòng tương ứng với một gói cước được đề xuất), việc join trực tiếp sang các bảng khác sẽ gây **nhân dòng và làm sai lệch kết quả phân tích**.

- **Cách xử lý:** tạo ra các **đặc trưng tổng hợp theo ID** nhằm gom toàn bộ thông tin của khách hàng về một dòng và mô tả đầy đủ hành vi đề xuất – chấp nhận gói cước.
- **Các đặc trưng mới bao gồm:**
  + **number_of_suggestions**: số lần được đề xuất (tổng số dòng theo ID)
  + **number_of_plans**: số gói khác nhau được đề xuất
  + **number_of_accept**: tổng số gói được khách hàng chấp nhận.
  + **number_of_accept_2**: tổng số gói khác nhau mà khách hàng chấp nhận đăng ký.
  + **mobile_plan**: liệt kê tên các gói unique khách hàng đã đăng
ký, nếu không đăng ký gói nào thì nhận giá trị **Unknown**.

In [ ]:
# Tạo bảng tổng hợp theo ID
mobile_plan_agg = mobile_plan_user.groupby('id').agg(
    number_of_suggestions = ('mobile_plan', 'count'),
    number_of_plans = ('mobile_plan', 'nunique'),
    number_of_accept = ('accept', 'sum'),
    number_of_accept_2 = ('mobile_plan',
                          lambda x: x[mobile_plan_user.loc[x.index, 'accept'] == 1].nunique()),
    mobile_plan = ('mobile_plan',
                   lambda x: ', '.join(
                       x[mobile_plan_user.loc[x.index, 'accept'] == 1].dropna().unique()
                   ) if x[mobile_plan_user.loc[x.index, 'accept'] == 1].nunique() > 0 else 'Unknown')
).reset_index()

mobile_plan_agg

,id,number_of_suggestions,number_of_plans,number_of_accept,number_of_accept_2,mobile_plan
0,1000.0,4,3,1.0,1,DATASILVER
1,1001.0,3,2,1.0,1,SOCIALMEDIA
2,1002.0,1,1,1.0,1,SOCIALMEDIAGOLD
3,1004.0,1,1,0.0,0,Unknown
4,1005.0,3,3,0.0,0,Unknown
...,...,...,...,...,...,...
11567,19994.0,1,1,0.0,0,Unknown
11568,19996.0,4,4,1.0,1,DATACALL
11569,19997.0,3,3,1.0,1,DATASILVER
11570,19998.0,3,3,0.0,0,Unknown


##  **4.Bảng mobile_plan_attr**

In [ ]:
mobile_plan_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mobile_plan  5 non-null      object
 1   description  5 non-null      object
 2   price        5 non-null      int64 
 3   duration     5 non-null      object
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes


In [ ]:
mobile_plan_attr

,mobile_plan,description,price,duration
0,DATASILVER,2GB/ day; high speed,100000,5d
1,DATAGOLD,5GB/ day; high speed,200000,5d
2,SOCIALMEDIA,"1 GB/day; Unlimited for Tik Tok, Facebook, You...",150000,3d
3,SOCIALMEDIAGOLD,"3GB/ day; high speed; Unlimited for Tik Tok, F...",250000,3d
4,DATACALL,2GB/ day; high speed; 300 mins call for extern...,200000,5d


**Remark:**
- Quan sát thấy, bảng không có null & duplicates và không có vấn đề xử lý chuẩn hóa.



In [ ]:
# Tiến hành join 3 bảng user.csv, context.csv, mobile_plan_agg sau khi tiền xử lý lại với nhau
data_train_clean = pd.merge(user, context, on='id', how='left')
# Join với mobile_plan_user_agg
data_train_clean = pd.merge(data_train_clean, mobile_plan_agg, on='id', how='left')

data_train_clean

,id,name,gender,age,education,profession,income,living_with,nation,phone,...,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,number_of_suggestions,number_of_plans,number_of_accept,number_of_accept_2,mobile_plan
0,11156,Rachel Gibbs,Female,21,Unknown,Unemployed,9.384000e+08,Unmarried_2,Australia,(08)-8012-7556,...,0,0,0,1,0,3,2,1.0,1,DATASILVER
1,4297,Karen Anderson,Other,22,Unknown,Unemployed,9.840000e+08,Unmarried_2,England,943-646-5203,...,1,0,0,1,0,4,3,1.0,1,SOCIALMEDIAGOLD
2,13301,김지원,Female,24,Unknown,Unemployed,1.063200e+09,Unmarried_1,Korea,010-4500-9888,...,1,0,0,1,0,4,3,1.0,1,DATASILVER
3,9920,Elisabeth Wähner,Female,24,Unknown,Unemployed,1.065600e+09,Unmarried_1,Denmark,+49(0) 587406963,...,1,0,0,1,30,2,2,1.0,1,SOCIALMEDIAGOLD
4,8424,Sra. Maria Luiza Nogueira,Female,25,Unknown,Unemployed,1.100000e+09,Unmarried_1,Brazil,(084) 0568 1445,...,1,0,0,1,0,2,2,1.0,1,DATASILVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11567,11602,Austin Barber,Male,26,Bachelor,Sales & Related,1.968000e+09,Single_0,England,001-798-297-3345x825,...,1,0,0,1,0,5,4,1.0,1,SOCIALMEDIA
11568,5645,남상철,Male,26,Bachelor,Sales & Related,2.013600e+09,Single_0,Korea,043-900-0635,...,1,0,1,0,0,2,2,0.0,0,Unknown
11569,15477,John Blanchard,Male,30,Bachelor,Sales & Related,1.838400e+09,Single_0,Us,230-283-6789x89147,...,0,0,1,0,13,2,2,0.0,0,Unknown
11570,8885,Troy Green,Male,30,Bachelor,Sales & Related,2.087500e+09,Single_0,Australia,+61.472.119.694,...,0,1,0,1,0,2,2,0.0,0,Unknown


In [ ]:
# Xuất file dữ liệu làm sạch sau khi join
data_train_clean.to_csv("data_train_cleaned.csv", index = False)